In [1]:
import os, sys, importlib

import geopandas as gpd
import pandas as pd

import shapely.ops
from shapely.geometry import Point, LineString
from shapely import ops

from osgeo import ogr


In [2]:
os.getcwd()

'C:\\Users\\lordBen\\Documents\\GitHub\\GOST_PublicGoods\\Implementations\\Canal - vector segmentation'

In [3]:
baseFolder = os.path.join(os.getcwd(), "SampleData","Canals_sample_3")
#This points to the input lines file - points to sample data right now
inLines = os.path.join(baseFolder, 'Canals_sample_3.shp')
#Enter the folder where you want output created
unqID = "UID"
inL = gpd.read_file(inLines)
#inL = inL.to_crs({'init':'epsg:3857'}) #need to project to a CRS with metres 
inL.head()

,CHANNEL_NA,CHANNEL_TY,SHAPE_Leng,Head_X,Head_y,Tail_X,Tail_Y,LENGTH,Shape_Le_1,UID,geometry
0,Jandraka Disty,D,29194.000197,73.498880,31.024473,73.259512,30.951668,29194.000197,29194.000197,10008000000,"LINESTRING (356722.854 3433281.631, 356407.332..."
1,Jassoki Kalasan Minor,M,5303.319279,73.478236,31.050279,73.433809,31.063130,5303.319279,5303.319279,10008002000,"LINESTRING (354791.383 3436168.912, 354793.727..."
2,Baran Pur Minor,M,10890.999684,73.487066,31.040016,73.397662,30.990222,10890.999684,10890.999684,10008001000,"LINESTRING (355618.574 3435019.745, 355479.722..."
3,Joya Disty,D,23415.233457,73.516364,31.035883,73.389395,30.879284,23415.233457,23415.233457,10007000000,"LINESTRING (358408.692 3434523.990, 358545.533..."
4,Bibi Pur Minor,M,3374.505853,73.458213,30.930921,73.470638,30.902517,3374.505853,3374.505853,10007001000,"LINESTRING (352697.238 3422965.144, 353072.296..."


In [4]:
def generate_subset_line(allCoords, length):
    #Generate the 
    allPts = []
    for cIdx in range(0, len(allCoords)):
        cPoint = Point(allCoords[cIdx])
        allPts.append(cPoint)
        if cIdx > 0:
            curLine = LineString(allPts)
            if curLine.length > length:
                break
    return(curLine)

def getBiggestString(geom):
    ''' get the longest segment from multiline string'''
    if geom.type != "MultiLineString":
        return(geom)
    longestSection = 0
    for x in geom:
        if x.length > longestSection:
            longest = x
            longestSection = x.length
    return(longest)

In [5]:
start = []
middle = []
end = []
riverNameColumn = unqID
inL['startPer'] = 0
inL['endPer'] = 0
for idx, row in inL.iterrows():
    x = row['geometry']
    if x.geom_type == 'MultiLineString':
        x = ops.linemerge(x)
    x = ogr.CreateGeometryFromWkt(x.wkt)
    if x.GetGeometryType() == 2:
        totalLength = x.Length()
        x.Segmentize(round(totalLength/500))    
        startLength = totalLength * 0.4
        endLength   = totalLength * 0.2
        allCoords = list(x.GetPoints())

        startLen = generate_subset_line(allCoords, startLength)
        allCoords.reverse()
        endLen   = generate_subset_line(allCoords, endLength)
        
        middleGeom = row['geometry'].difference(startLen.buffer(0.001,cap_style=2)).difference(endLen.buffer(0.001,cap_style=2))
        middleGeom = getBiggestString(middleGeom)
        
        start.append([row[riverNameColumn], startLen])
        end.append([row[riverNameColumn], endLen])
        middle.append([row[riverNameColumn], middleGeom])
        
        inL['startPer'].loc[idx] = startLen.length/totalLength
        inL['endPer'].loc[idx] = endLen.length/totalLength
        inL['geometry'].loc[idx] = LineString(allCoords)
        
        print("%s - %s - %s - %s" % (row[riverNameColumn], x.Length(), startLen.length, endLen.length))
    else:
        break
startDF = pd.DataFrame(start, columns=["NAME", "GEOM"])
endDF = pd.DataFrame(end,  columns=["NAME", "GEOM"])
middleDF = pd.DataFrame(middle, columns=["NAME", "GEOM"])

C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


10008000000 - 29194.000197278416 - 11696.196811054238 - 5881.21741490273
10008002000 - 5303.31927925844 - 2129.0916204691393 - 1066.0262993209142


C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


10008001000 - 10890.999684395832 - 4358.955907569318 - 2194.2714067931097


C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


...

10001000000 - 2023.0671497643157 - 811.2952841855997 - 405.5473061745798
10004001000 - 4135.282115430735 - 1654.7020939983956 - 832.0573397305884


C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


10005000000 - 3695.6605096609437 - 1480.2722944689579 - 741.0842639849272


C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\Users\lordBen\AppData\Local\conda\conda\envs\Geog36\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


10006000000 - 8535.470579116549 - 3420.645927682937 - 1716.041535760671
10006001000 - 3799.052380245794 - 1523.4508627133775 - 766.2408423354384


In [6]:
from shapely.wkt import loads

startDF_geom = startDF['GEOM']
startDF = startDF.drop(['GEOM'], axis=1)
startGDF = gpd.GeoDataFrame(startDF, geometry=startDF_geom, crs=inL.crs)

endDF_geom = endDF['GEOM']
endDF = endDF.drop(['GEOM'], axis=1)
endGDF = gpd.GeoDataFrame(endDF, geometry=endDF_geom, crs=inL.crs)

middleDF_geom = middleDF['GEOM']
middleDF = middleDF.drop(['GEOM'], axis=1)
middleDF = gpd.GeoDataFrame(middleDF, geometry=middleDF_geom, crs=inL.crs)


In [7]:
'''startDF.to_csv(os.path.join(baseFolder,"canal_starts.csv"))
endDF.to_csv(os.path.join(baseFolder,"canal_ends.csv"))
inL.to_csv(os.path.join(baseFolder,"canals_with_length.csv"))
'''
startGDF.to_file(os.path.join(baseFolder,"canal_starts.shp"))
endGDF.to_file(os.path.join(baseFolder,"canal_ends.shp"))
middleDF.to_file(os.path.join(baseFolder,"canal_middles.shp"))
inL.to_file(os.path.join(baseFolder,"canal_with_length.shp"))

# Buffering and clipping
For each start and end segment, the feature needs to be buffered, but the buffer needs to be limited to not cover anything covered by other canals

In [25]:
bufferDist = 1000 # metres
bufferSet = startDF # endDF
clipBuffers = False

for bufferSet, outName in [[startDF, 'canal_start_buffered_clipped'],[endDF, 'canal_end_buffered_clipped']]:
    bufferSet['GEOM2'] = ''
    outFile = os.path.join(baseFolder,"%s.shp" % outName)
    for idx, row in bufferSet.iterrows():
        curName = row['NAME']
        curShape = row['geometry']        
        bufferShape = curShape.buffer(bufferDist, cap_style=2, join_style=2, mitre_limit=1000000)

        if clipBuffers:
            #Select canals that intersect current buffered shape
            intersecting_canals = inL[inL.intersects(bufferShape)]
            if intersecting_canals.shape[0] > 1:
                intersecting_canals = intersecting_canals[intersecting_canals[riverNameColumn] != curName]        
                for selCanal in intersecting_canals['geometry']:
                    xx = shapely.ops.split(bufferShape, selCanal)
                    maxArea = 0            
                    selArea = None
                    for x in xx:
                        if x.area > maxArea:
                            maxArea = x.area
                            selArea = x
                    bufferShape = selArea
        bufferSet['GEOM2'].iloc[idx] = bufferShape
    bufferSet_geom = bufferSet['GEOM2']
    bufferSetGDF = bufferSet.drop('GEOM2', axis=1)
    try:
        bufferSetGDF = bufferSetGDF.drop('GEOM', axis=1)
    except:
        pass
    bufferSetGDF = gpd.GeoDataFrame(bufferSetGDF, geometry=bufferSet_geom, crs=inL.crs)
    bufferSetGDF.to_file(outFile)
    #bufferSet.to_csv(outFile)

In [11]:
curShape.buffer?